In [1]:
import ast  # Add this at the beginning of your script
import json
from openai import AzureOpenAI
from typing import List, Dict

# pip install python-docx
from docx import Document
from docx.shared import Inches

In [2]:
api_version = "2023-07-01-preview"
api_key = ''
api_endpoint = ''
deployment_model_name = ''
deployment_model_name_long = 'GPT4-32k'

chat_client  = AzureOpenAI(
    api_version=api_version,
    api_key = api_key,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=api_endpoint,
    # Navigate to the Azure OpenAI Studio to deploy a model.
    azure_deployment=deployment_model_name,  # e.g. gpt-35-instant
)

chat_client_long  = AzureOpenAI(
    api_version=api_version,
    api_key = api_key,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=api_endpoint,
    # Navigate to the Azure OpenAI Studio to deploy a model.
    azure_deployment=deployment_model_name_long,  # e.g. gpt-35-instant
)

def synthesize_text(prompt: str) -> str:
    messages =[
        {
            "role": "system", "content": "You are a helpful assistant whom is imaginative, witty, creative, and intelligent.",
            "role": "user", "content": prompt,
        },
    ]

    completion = chat_client.chat.completions.create(
        model="<ignored>",
        messages=messages,
        temperature=.5,
        max_tokens = 4096
    )
    return completion.choices[0].message.content.strip()

def synthesize_text_long(prompt: str) -> str:
    messages =[
        {
            "role": "system", "content": "You are a helpful assistant whom is imaginative, witty, creative, and intelligent.",
            "role": "user", "content": prompt,
        },
    ]

    completion = chat_client.chat.completions.create(
        model="<ignored>",
        messages=messages,
        temperature=.7,
        max_tokens = 16000
    )
    return completion.choices[0].message.content.strip()

In [3]:
class StoryIdea:
    def __init__(self, genre, themes, mood, premise):
        self.genre = genre
        self.themes = themes
        self.mood = mood
        self.premise = premise

class Character:
    def __init__(self, name, age, role, traits, backstory):
        self.name = name
        self.age = age
        self.role = role
        self.traits = traits
        self.backstory = backstory

In [4]:
# examples below 
my_story_idea = StoryIdea(
    genre="Adventure",
    themes=["Friendship", "Courage", "Self-discovery"],
    mood="Exciting",
    premise="A group of teenagers discovers a hidden world while on a summer camp."
)
        
protagonist = Character(
    name="Alex",
    age=16,
    role="Protagonist",
    traits=["Brave", "Curious", "Compassionate"],
    backstory="Grew up in a small town, always dreamed of adventure."
)

antagonist = Character(
    name="Morpheus",
    age='undefined',
    role="Antagonist",
    traits=["Mysterious", "Powerful", "Manipulative"],
    backstory="Banished from the hidden world, seeks revenge."
)

plot_points = [
    {"chapter": 1, "setting": "Small town", "events": ["Summer camp begins", "Alex meets new friends"], "twist": "They find a mysterious map"},
    {"chapter": 2, "setting": "Forest", "events": ["They follow the map", "Encounter strange creatures"], "twist": "The map leads to a hidden entrance"},
    # Add more chapters as needed
]

storyboard = {
    1: ["Introduction of characters", "Discovery of the map"],
    2: ["Journey into the forest", "First encounter with the hidden world"],
    # Add more chapters as needed
}

chapter_outlines = {
    1: {
        "scenes": ["Alex's home life", "First day of camp"],
        "character_developments": ["Alex shows courage", "New friendships form"],
        "plot_advancements": ["Discovery of the map"]
    },
    2: {
        "scenes": ["Exploration of the forest", "Encounter with a mystical creature"],
        "character_developments": ["Alex's leadership", "Teamwork among the group"],
        "plot_advancements": ["Entrance to the hidden world found"]
    },
    # Add more chapters as needed
}


In [5]:
# Automate the synthesis of story elements
def generate_story_idea() -> StoryIdea:
    prompt = "Generate a story idea with genre|themes|mood|premise for a young adult novel. Use the | as the separator character for genre themes mood and premise. Use a comma separator for multiple themes."
    response = synthesize_text(prompt)
    
    # Split the response by commas and ensure all parts are present
    parts = response.split('|')
    if len(parts) < 4:  # Checking if all parts are present
        print("Incomplete response received:", response)
        return StoryIdea(genre="Unknown", themes=[], mood="Unknown", premise="Unknown")

    # Process each part of the response
    genre = parts[0].split(': ')[1] if len(parts[0].split(': ')) > 1 else "Unknown"
    themes = parts[1].split(': ')[1].split(', ') if len(parts[1].split(': ')) > 1 else []
    mood = parts[2].split(': ')[1] if len(parts[2].split(': ')) > 1 else "Unknown"
    premise = parts[3].split(': ')[1] if len(parts[3].split(': ')) > 1 else "Unknown"
    
    return StoryIdea(genre=genre, themes=themes, mood=mood, premise=premise)

def parse_narrative_to_plot_points(narrative: str) -> List[Dict]:
    # Split the response by "Chapter" to separate out individual plot points
    chapters = narrative.split('Chapter ')[1:]  # Remove the first split part which is empty or unrelated text
    plot_points = []

    for chapter in chapters:
        # Further split each chapter part by "Setting", "Main Events", and "Twist"
        parts = chapter.split('\n')
        if len(parts) < 4:
            continue  # Skip if there aren't enough parts

        # Extract chapter number, setting, main events, and twist from the parts
        chapter_number = parts[0].split(':')[0].strip()
        setting = parts[1].split('-')[1].strip() if '-' in parts[1] else "Unknown"
        main_events = parts[2].split(': ')[1].strip() if ': ' in parts[2] else "Unknown"
        twist = parts[3].split(': ')[1].strip() if ': ' in parts[3] else "Unknown"

        # Create a dictionary for the plot point and add it to the list
        plot_point = {
            'chapter': chapter_number,
            'setting': setting,
            'events': [main_events],  # Wrapping main events in a list for consistency
            'twist': twist
        }
        plot_points.append(plot_point)

    return plot_points
	
# Then, in your generate_plot_points function, you could use this parser:
def generate_plot_points(story_idea: StoryIdea, n: int) -> List[Dict]:
    prompt = (f"Based on a {story_idea.genre} genre with themes of {', '.join(story_idea.themes)} and an overall {story_idea.mood} mood, "
              f"and the premise '{story_idea.premise}', generate {n} key plot points for a young adult novel. "
              "Each plot point should include a chapter number, setting, main events, and a twist.")

    response = synthesize_text(prompt)

    plot_points = []  # Initialize an empty list to store plot points
    # Split the response into separate chapters, assuming they start with '1. Chapter', '2. Chapter', etc.
    chapters = response.split('\n\n')  # Splitting chapters assuming they are separated by double newlines
    for chapter in chapters:
        lines = chapter.strip().split('\n')
        if len(lines) >= 5:  # Expecting at least 5 lines: chapter indicator, setting, main events, and twist
            # Extract and process chapter number, setting, main events, and twist from the lines
            chapter_number = lines[0].split(':')[0].split('. ')[1].strip()  # Splitting by ':' and '. ' to extract number
            setting = lines[1].split(':')[1].strip()
            main_events = lines[2].split(':')[1].strip()
            twist = lines[3].split(':')[1].strip()
            plot_point = {
                'chapter': chapter_number,
                'setting': setting,
                'events': [main_events],  # Wrap main events in a list
                'twist': twist
            }
            plot_points.append(plot_point)

    return plot_points


def generate_storyboard(story_idea: StoryIdea, plot_points: List[Dict], characters: List[Character], num_chapters: int) -> Dict:
    # Construct a description from the story idea
    genre = story_idea.genre
    themes = ", ".join(story_idea.themes)
    mood = story_idea.mood
    premise = story_idea.premise
    story_description = f"A {genre} story with themes of {themes}, set in a {mood} mood, with the premise: '{premise}'."

    # Summarize characters for the storyboard context
    character_summaries = '. '.join([f"{character.name}, a {character.role} with traits such as {' and '.join(character.traits)}, has a backstory of {character.backstory}" for character in characters])

    # Summarize plot points
    plot_point_summaries = '. '.join([f"Chapter {point['chapter']}: Set in {point['setting']}, where {'; '.join(point['events'])}. Twist: {point['twist']}" for point in plot_points])

    # Construct the prompt for the storyboard
    prompt = (f"Based on the story context: {story_description} And characters: {character_summaries}. "
              f"Create a storyboard outlining the key events for each of the first {num_chapters} chapters based on the following plot points: {plot_point_summaries}.")

     # Generate the storyboard
    response = synthesize_text(prompt)

    # Instead of trying to decode JSON, split the response into chapters and parse manually
    storyboard = {}
    chapters = response.split('Chapter ')[1:]  # Splitting each chapter into parts

    for chapter_info in chapters:
        # Split each chapter information into lines
        parts = chapter_info.split('\n')
        if len(parts) < 4:  # Making sure we have all the parts
            continue  # If not, skip this chapter

        # Extract information from the parts
        chapter_number = parts[0].split(':')[0].strip()  # Assuming the first part is 'Chapter X:'
        setting = parts[1].split(': ')[1].strip() if len(parts[1].split(': ')) > 1 else "Unknown"
        events = parts[2].strip().split('-')[1:]  # Assuming events are listed with '- '
        twist = parts[3].split(': ')[1].strip() if len(parts[3].split(': ')) > 1 else "Unknown"

        # Construct a storyboard entry
        storyboard[chapter_number] = {
            'setting': setting,
            'events': events,
            'twist': twist
        }

    return storyboard

# Function to identify required characters from the story idea or plot points
def identify_required_characters(plot_points: List[Dict], additional_roles: List[str]) -> List[str]:
    roles = set(additional_roles)  # Starting with additional roles like 'Protagonist', 'Antagonist'
    for point in plot_points:
        # This is simplistic; real stories may need more nuanced character role identification
        if "events" in point:
            for event in point["events"]:
                if "meets" in event or "confronts" in event:
                    # Assuming the format is like "Protagonist meets [CharacterRole]"
                    role = event.split()[-1].strip('[]')  # Extracts the role from the event
                    roles.add(role)
    return list(roles)

# Function to generate multiple characters based on their roles
def generate_characters(roles: List[str]) -> List[Character]:
    characters = []
    for role in roles:
        prompt = f"Create a character description for a {role} in a young adult novel including name, age, personality traits, and backstory."
        response = synthesize_text(prompt)
        # Split the response and check if all parts are present
        parts = response.split(', ')
        
        if len(parts) < 4:  # Checking if all parts are present
            print("Incomplete response received:", response)
            # Provide default values in case of incomplete response
            character = Character(
                name="Unknown",
                age="Unknown",
                role=role,
                traits=["Unknown"],  # Default to a list with a single string 'Unknown'
                backstory="Unknown"
            )
        else:
            # If all parts are present, process and return the Character object
            name = parts[0].split(': ')[1] if len(parts[0].split(': ')) > 1 else "Unknown"
            age = parts[1].split(': ')[1] if len(parts[1].split(': ')) > 1 else "Unknown"
            traits = parts[2].split(': ')[1].split(', ') if len(parts[2].split(': ')) > 1 else ["Unknown"]
            backstory = parts[3].split(': ')[1] if len(parts[3].split(': ')) > 1 else "Unknown"
            character = Character(name=name, age=age, role=role, traits=traits, backstory=backstory)
        
        characters.append(character)
    return characters


def generate_chapter_outlines_with_titles(story_idea: StoryIdea, plot_points: List[Dict], characters: List[Character], storyboard: Dict, num_chapters: int) -> Dict:
    # Construct a summary from the story idea
    genre = story_idea.genre
    themes = ", ".join(story_idea.themes)
    mood = story_idea.mood
    premise = story_idea.premise
    story_description = f"A {genre} story focused on themes such as {themes}, set in a {mood} mood. Premise: '{premise}'."

    # Summarize characters for the outlines context
    character_summaries = '. '.join([f"{character.name}, a {character.role}, characterized by {' and '.join(character.traits)} with a backstory of {character.backstory}" for character in characters])

    # Summarize plot points
    plot_point_summaries = '. '.join([f"Chapter {point['chapter']}: In {point['setting']}, {'; '.join(point['events'])}. Twist: {point['twist']}" for point in plot_points])

    # Construct and collect chapter outlines
    chapter_outlines_with_titles = {}
    for chapter_num in range(1, num_chapters + 1):
        # Here, define the 'prompt' for each chapter based on the given story idea and other parameters
        prompt = (f"Generate an outline for Chapter {chapter_num} based on the story genre '{genre}', "
                  f"themes '{themes}', mood '{mood}', and premise '{premise}'. "
                  f"Summarize the chapter including introduction, scenes, and conclusion. "
                  f"Incorporate elements from the story context: {story_description} and characters: {character_summaries}. "
                  f"Ensure the outline reflects plot points: {plot_point_summaries}.")

        # Generate the chapter outline using the defined 'prompt'
        response = synthesize_text(prompt)

        # Process the text response directly
        outline_sections = response.split('\n\n')  # Assuming each section is separated by a double newline
        chapter_outline = {
            'Introduction': None,
            'Scenes': [],
            'Conclusion': None
        }
        current_section = None
        for section in outline_sections:
            if section.startswith('I.'):
                current_section = 'Introduction'
                chapter_outline[current_section] = section
            elif section.startswith('II.') or section.startswith('III.') or section.startswith('IV.') or section.startswith('V.'):
                current_section = 'Scenes'
                chapter_outline[current_section].append(section)
            elif section.startswith('VI.'):
                current_section = 'Conclusion'
                chapter_outline[current_section] = section
        
        # Generate a simple title for each chapter based on its main setting or event
        chapter_title = f"Chapter {chapter_num}: {storyboard.get(str(chapter_num), {}).get('setting', 'Unknown').split()[0]} Mystery"
        
        # Ensure existing outline information is included if it exists
        existing_outline = chapter_outlines.get(str(chapter_num), {})  # Safely get the chapter outline
        chapter_outlines_with_titles[str(chapter_num)] = {
            'title': chapter_title,
            **existing_outline  # Safely include existing outline information
        }

    return chapter_outlines_with_titles

def generate_novel_text(story_idea: StoryIdea, plot_points: List[Dict], characters: List[Character], storyboard: Dict, chapter_outlines: Dict) -> Dict[int, str]:
    # Construct a summary from the story idea
    genre = story_idea.genre
    themes = ", ".join(story_idea.themes)
    mood = story_idea.mood
    premise = story_idea.premise
    story_description = f"This is a {genre} story with themes such as {themes}, set in a {mood} mood. The premise is '{premise}'."

    # Summarize characters for the novel context
    character_summaries = '. '.join([f"{character.name}, a {character.role}, characterized by traits such as {' and '.join(character.traits)}, with a backstory of {character.backstory}" for character in characters])

    novel_text = {}
    for chapter_num, outline in chapter_outlines.items():
        # Safely access the 'scenes', 'character_developments', and 'plot_advancements' keys
        scenes = ' '.join(outline.get('Scenes', []))  # Default to an empty list if 'Scenes' is not found
        character_developments = outline.get('Introduction', 'No introduction provided')  # Assuming you meant 'Introduction' as developments
        plot_advancements = outline.get('Conclusion', 'No conclusion provided')  # Assuming 'Conclusion' refers to plot advancements

        # The rest of your code for generating text remains the same
        # Incorporate the storyboard and plot points for the chapter...
        storyboard_summary = storyboard.get(str(chapter_num), "No specific events outlined yet.")
        chapter_plot_points = next((point for point in plot_points if str(point["chapter"]) == str(chapter_num)), None)
        plot_point_summary = f"In {chapter_plot_points['setting']}, {'; '.join(chapter_plot_points['events'])}. Twist: {chapter_plot_points['twist']}" if chapter_plot_points else "No plot points specified."

        # Construct the narrative prompt for the chapter...
        prompt = (f"Based on the overall story context: {story_description} and characters: {character_summaries}, "
                  f"write the narrative for Chapter {chapter_num}. The chapter is based on the following outline: Scenes: {scenes}. "
                  f"Character developments: {character_developments}. Plot advancements: {plot_advancements}. "
                  f"Storyboard suggests: {storyboard_summary}. Plot points detail: {plot_point_summary}")

        # Generate the chapter narrative
        chapter_text = synthesize_text_long(prompt)
        novel_text[int(chapter_num)] = chapter_text

    return novel_text

def create_novel_document(novel_text: Dict[int, str], title: str, filename: str):
    doc = Document()
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(0.5)
        section.bottom_margin = Inches(0.5)
        section.left_margin = Inches(0.5)
        section.right_margin = Inches(0.38)
        section.page_width = Inches(6)
        section.page_height = Inches(9)

    doc.add_heading(title, 0)  # Use the title for the document

    for chapter_num, text in sorted(novel_text.items()):
        # Use the title from the chapter outlines
        chapter_title = chapter_outlines.get(str(chapter_num), {}).get('title', f'Chapter {chapter_num}')
        doc.add_heading(chapter_title, level=1)
        doc.add_paragraph(text)
        if chapter_num != max(novel_text.keys()):
            doc.add_page_break()

    doc.save(filename)
    print(f'Novel saved as {filename}.')
    
def generate_title_from_storyboard(story_idea: StoryIdea, storyboard: Dict) -> str:
    # Generate a summary from the storyboard
    storyboard_summary = '. '.join([
        f"Chapter {chapter_num}: {details.get('setting', 'Unknown')} - " +
        f"{details.get('events', ['Unknown'])[0] if details.get('events') else 'Unknown'} - " +
        f"Twist: {details.get('twist', 'Unknown')}" 
        for chapter_num, details in storyboard.items()
    ])

    # Construct a title based on the genre, mood, and a key element from the premise or plot points
    prompt = (f"Based on the overall story idea: Genre - {story_idea.genre}, Themes - {', '.join(story_idea.themes)}, Mood - {story_idea.mood}, and storyboard: {storyboard_summary}, "
              f"write a title for a book. Only create the title name and no jargon.")

    # Generate the chapter narrative
    title = synthesize_text(prompt)
    return title

In [ ]:
new_story_idea = generate_story_idea()
new_plot_points = generate_plot_points(new_story_idea, 20)  # 15 - 25 is optimal number of plot points for a young adult novel

required_roles = identify_required_characters(new_plot_points, ['Protagonist', 'Antagonist'])
characters = generate_characters(required_roles)

num_chapters = 30  # Or however many chapters you want to storyboard initially, 20 - 40 is optmial
new_storyboard = generate_storyboard(new_story_idea, new_plot_points, characters, num_chapters)

# After generating the storyboard:
new_title = generate_title_from_storyboard(new_story_idea, new_storyboard)
print(f"Generated Title: {new_title}")

new_chapter_outlines_with_titles = generate_chapter_outlines_with_titles(new_story_idea, new_plot_points, characters, new_storyboard, num_chapters)

In [ ]:
# Generate the text for each chapter of the novel
complete_novel_text = generate_novel_text(new_story_idea, new_plot_points, characters, new_storyboard, new_chapter_outlines_with_titles)

In [ ]:
complete_novel_text

In [ ]:
# When creating the novel document:
filename = f"{new_title.replace(' ', '_').lower()}.docx"
create_novel_document(complete_novel_text, new_title, filename)